In [0]:
# this is the dataset to use
# https://github.com/ofrendo/WebDataIntegration/blob/7db877abadd2be94d5373f5f47c8ccd1d179bea6/data/goldstandard/forbes_freebase_goldstandard_train.csv

In [0]:
!wget https://raw.githubusercontent.com/ofrendo/WebDataIntegration/7db877abadd2be94d5373f5f47c8ccd1d179bea6/data/goldstandard/forbes_freebase_goldstandard_train.csv

--2020-01-30 16:15:53--  https://raw.githubusercontent.com/ofrendo/WebDataIntegration/7db877abadd2be94d5373f5f47c8ccd1d179bea6/data/goldstandard/forbes_freebase_goldstandard_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7324 (7.2K) [text/plain]
Saving to: ‘forbes_freebase_goldstandard_train.csv’

forbes_freebase_gol 100%[===================>]   7.15K  --.-KB/s    in 0s      

2020-01-30 16:15:53 (213 MB/s) - ‘forbes_freebase_goldstandard_train.csv’ saved [7324/7324]



In [0]:
import pandas as pd

data = pd.read_csv('forbes_freebase_goldstandard_train.csv')

loading training data

In [0]:
!wget https://raw.githubusercontent.com/ofrendo/WebDataIntegration/7db877abadd2be94d5373f5f47c8ccd1d179bea6/data/goldstandard/forbes_freebase_goldstandard_test.csv

--2020-01-30 16:19:47--  https://raw.githubusercontent.com/ofrendo/WebDataIntegration/7db877abadd2be94d5373f5f47c8ccd1d179bea6/data/goldstandard/forbes_freebase_goldstandard_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 976 [text/plain]
Saving to: ‘forbes_freebase_goldstandard_test.csv’

forbes_freebase_gol 100%[===================>]     976  --.-KB/s    in 0s      

2020-01-30 16:19:47 (200 MB/s) - ‘forbes_freebase_goldstandard_test.csv’ saved [976/976]



In [0]:
data.head()

,General Electric,General Electric.1,TRUE
0,Wells Fargo,Wells Fargo,True
1,Bank of China,Industrial and Commercial Bank of China (Asia),True
2,PetroChina,PetroChina,True
3,Apple,Apple Inc.,True
4,Citigroup,Citigroup,True


In [0]:
len(data)

212

In [0]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144668 sha256=1ec94057e46fbb71326db91a3e18b68e799ebb8b36e245ac6b79597763043eb8
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [0]:
import joblib
import os
from multiprocessing import Pool
import re
from difflib import SequenceMatcher  # for longest common substring
from functools import partial
from operator import itemgetter
import Levenshtein  # levenstein/edit distance; docs here: https://rawgit.com/ztane/python-Levenshtein/master/docs/Levenshtein.html

In [0]:
def clean_string(string):
    '''We will use this functions to remove special characters etc before 
    any string distance calculation.
    '''
    return ''.join(map(lambda x: x.lower() if str.isalnum(x) else ' ', string)).strip()

I think it's useful to make a distinction here
- distance functions between two strings
- string featurization

The string distances we can use to establish a baseline performance. But with small changes we can make them string featurization functions and use them in classifier functions in a machine learning approach. However, let us get first to string distances.


In [0]:
def levenstein_distance(s1_, s2_):
    s1, s2 = clean_string(s1_), clean_string(s2_)
    len_s1, len_s2 = len(s1), len(s2)
    return Levenshtein.distance(
        s1, s2
    ) / max([len_s1, len_s2])

def jaro_winkler_distance(s1_, s2_):
    s1, s2 = clean_string(s1_), clean_string(s2_)
    return Levenshtein.jaro_winkler(s1, s2)

def common_substring_similarity(s1_, s2_):
    s1, s2 = clean_string(s1_), clean_string(s2_)
    match = SequenceMatcher(None, s1, s2).find_longest_match(0, len_s1, 0, len_s2)
    len_s1, len_s2 = len(s1), len(s2)
    norm = max([len_s1, len_s2])
    return min([1, match.size / norm])

In [0]:
# we can use these string functions for fuzzy string match
# some matches are not very good, so we should make sure, we have thresholds.

pool = Pool(50)

def fuzzy_string_search(
    s1, string_list,
     string_compare,
      threshold=lambda sim: sim>0.7
    ):
    '''Search through a list of strings using a string_comparison function
    in order to find the best match.

    Parameters:
    - s1: string to search for
    - string_list: list of strings
    - string_compare: string comparison function to return a similarity or a 
      distance.
    - threshold: cut-off function to decide if returning the best match or
      nothing at all. This threshold function has to take into account if we
      are using a string similarity or a string distance.

    Return the best matching string if threshold reached.
    Otherwise return None.

    Example:
    >> company_list = [
        'Blackrock',
        'Credit Suisse',
        'Goldman Sachs',
        'Bank of America/Meryll Lynch',
        'Morgan Stanley',
        'LEK',
        'JP Morgan',
        'Nomura',
        'BNP Paribas',
        'WPP',
        'Rothschild',
        'Allianz',
    ]
    >> fuzzy_string_search(
      'SAP',
      company_list,
      jaro_winkler_distance,
      threshold=lambda dist: dist<0.7
    )
    '''
    string_match = partial(string_compare, s2_=s1)
    comparisons = pool.map(string_match, string_list)
    index, element = max(enumerate(comparisons), key=itemgetter(1))
    #print(f"match {string_list[index]} with score {element}")
    if threshold(element):
        return string_list[index]
    else:
        return None

In [0]:
company_list = [
        'Blackrock',
        'Credit Suisse',
        'Goldman Sachs',
        'Bank of America/Meryll Lynch',
        'Morgan Stanley',
        'LEK',
        'JP Morgan',
        'Nomura',
        'BNP Paribas',
        'WPP',
        'Rothschild',
        'Allianz',
    ]
fuzzy_string_search(
      'SAP',
      company_list,
      jaro_winkler_distance,
      threshold=lambda dist: dist<0.7
    )


'WPP'

In [0]:
# We can also use featurizations of strings, for example using sklearn
# inbuilt functionality such as CountVectorizers or TfidfVectorizer.


from sklearn.feature_extraction.text import CountVectorizer
# the CountVectorizer counts the occurences of features. These features
# can be composed of characters or words; we are interested in character-
# based features. We clean the strings as before and we take ngrams.
# Also try TfidfVectorizer for a baseline performance
ngram_featurizer = CountVectorizer(
    min_df=1,
    analyzer='char',
    ngram_range=(1,1),  # this is the range of ngrams that are to be extracted!
    preprocessor=clean_string
)
company_space = ngram_featurizer.fit_transform(company_list)

In [0]:
company_space

<12x24 sparse matrix of type '<class 'numpy.int64'>'
	with 96 stored elements in Compressed Sparse Row format>

In [0]:
# Some alternative way to featurize strings. These can be used in similar ways 
# to the CountVectorizer really. Apply these as a preprocessor to a classifier
# and check the performance in distinguishing between match and no-match.

def editops_featurizer(s1_, s2_):
    '''Counts the replace, insert and delete operations between two strings
    and normalizes these by maximum string length.

    This featurization could be interesting to find out which operation is
    most useful.
    '''
    s1, s2 = clean_string(s1_), clean_string(s2_)
    len_s1, len_s2 = len(s1), len(s2)
    ops = Levenshtein.editops(
        s1, s2
    )
    index_dict = {'insert': 0, 'replace': 1, 'delete': 2}
    features = np.zeros((3))
    for op in edit_ops:
        features[index_dict[op[0]]] += 1
    features / max([len_s1, len_s2])  
    return features

def common_substring_featurizer(s1_, s2_):
    '''Here we extract 1. the normalized length of the common substring
    and whether the common substring matches 2. the beginning or
    3. the end of a word.
    '''
    s1, s2 = clean_string(s1_), clean_string(s2_)
    len_s1, len_s2 = len(s1), len(s2)
    longer_string = s1 if len_s1 > len_s2 else s2
    norm = max([len_s1, len_s2])    
    match = SequenceMatcher(None, s1, s2).find_longest_match(0, len_s1, 0, len_s2)
    substring = s1[match.a: match.a + match.size]
    
    m1 = re.search(
        '(?:^|\s|[a-z])' + substring,
        longer_string
    )
    m2 = re.search(
        substring + '(?:[a-z]|\s|$)',
        longer_string
    )
    return min([1, match.size / norm]), m1 is not None, m2 is not None

In [0]:
# siamese network dimensionality reduction
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Lambda, Input
import tensorflow as tf
from tensorflow.keras import backend as K



def create_string_featurization_model(feature_dimensionality, output_dim=50):
    '''
    Use for string featurization in combination with siamese models.
    Just a non-linear projection as a way of reducing the feature dimensionality
    in a meaningful way.

    Parameters:
        feature_dimensionality - number of features coming from the vectorizer
          or string featurization function
        output_dim - dimensions of the embedding/projection that we are trying
          to create
    '''
    preprocessing_model = Sequential()
    preprocessing_model.add(
        Dense(output_dim, activation='selu', input_dim=feature_dimensionality)
    )
    preprocessing_model.summary()
    return preprocessing_model

def create_siamese_model(preprocessing_models, #initial_bias =
                          input_shapes=(10,)):
    def euclidean_distance(vects):
        x, y = vects
        x = K.l2_normalize(x, axis=-1)
        y = K.l2_normalize(y, axis=-1)
        sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
        return K.sqrt(K.maximum(sum_square, K.epsilon()))
    
    if not isinstance(preprocessing_models, (list, tuple)):
        raise ValueError('preprocessing models needs to be a list or tuple of models')

    print('{} models to be trained against each other'.format(len(preprocessing_models)))
    if not isinstance(input_shapes, list):
        input_shapes = [input_shapes] * len(preprocessing_models)
    
    inputs = []
    intermediate_layers = []
    for preprocessing_model, input_shape in zip(preprocessing_models, input_shapes):
        inputs.append(Input(shape=input_shape))
        intermediate_layers.append(preprocessing_model(inputs[-1]))

    layer_diffs = []
    for i in range(len(intermediate_layers)-1):        
        layer_diffs.append(
            Lambda(euclidean_distance)([intermediate_layers[i], intermediate_layers[i+1]])
        )    
    siamese_model = Model(inputs=inputs, outputs=layer_diffs)
    siamese_model.summary()
    return siamese_model

def compile_model(model):
    model.compile(
        optimizer='rmsprop',
        loss='mse',
        metrics=[
            #'accuracy',
            #tf.keras.metrics.FalseNegatives(name='fn'), 
            #tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            #tf.keras.metrics.AUC(name='auc'),
        ]
    )

# use like this:
feature_dims = len(ngram_featurizer.get_feature_names())
string_featurization_model = create_string_featurization_model(feature_dims, output_dim=50)

siamese_model = create_siamese_model(
    preprocessing_models=[string_featurization_model, string_featurization_model],
    input_shapes=[(feature_dims,), (feature_dims,)],
)
compile_model(siamese_model)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1250      
Total params: 1,250
Trainable params: 1,250
Non-trainable params: 0
_________________________________________________________________
2 models to be trained against each other
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 24)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)

In [ ]:
siamese_model.fit()

In [0]:
# some very basic plotting stuff
# maybe useful for training

import matplotlib.pyplot as plt
import matplotlib as mpl


def plot_metrics(history, metrics=['loss', 'auc', 'precision', 'recall']):
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        
    mpl.rcParams['figure.figsize'] = (12, 10)
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
        plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
        plt.ylim([0.8,1])
    else:
        plt.ylim([0,1])

    plt.legend()


#plot_metrics(history, metrics=['recall'])

In [0]:
!pip install annoy  # https://github.com/spotify/annoy

     |████████████████████████████████| 645kB 2.8MB/s 
  Created wheel for annoy: filename=annoy-1.16.2-cp36-cp36m-linux_x86_64.whl size=310418 sha256=9c0c67d0ed535901d016f5cbd0dfe87ce50949631eb69f250acfd494359b0d23
  Stored in directory: /root/.cache/pip/wheels/d3/d7/68/3795670ef4c6781fc10df0d6cf83b922244aa28cd9489d1176
Successfully built annoy


In [0]:
# given featurized strings, we can use nearest neighbor-search/classification
# to decide very quickly which string matches and how good it matches.

from annoy import AnnoyIndex

index = AnnoyIndex(company_space.shape[1], 'euclidean')
for i, emp in enumerate(company_space.toarray()):
    index.add_item(i, emp)
    
index.build(10)
index.save('string_list.ann')


v = ngram_featurizer.transform(['Meryll Lynch']).toarray().flatten()

ngram_featurizer.get_feature_names()

[' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'w',
 'y',
 'z']

In [0]:
index.get_nns_by_vector(v, 3, search_k=-1, include_distances=True)
ngram_featurizer.transform(['HSBC']).toarray()

array([[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0]])